In [16]:
import pandas as pd

# CSVファイルのパスを指定（例: 'data.csv' や '/content/sample.csv' に変更）
path = '../data/名簿一覧_id.csv'

# 読み込み（必要に応じて encoding, sep, dtype などを指定）
df = pd.read_csv(path)
# 表示
#df.head()

In [17]:
# 会社名から略称を取得する関数
def get_company_abbr(company_name):
    if pd.isna(company_name):
        return '未定'
    elif 'ソニー株式会社(SEC)' in company_name:
        return 'SEC'
    elif 'ソニーセミコンダクタソリューションズ株式会社(SSS)' in company_name:
        return 'SSS'
    elif 'ソニーグループ株式会社(SGC)' in company_name:
        return 'SGC'
    elif 'ソニー・インタラクティブエンタテインメント(SIE)' in company_name:
        return 'SIE'
    else:
        return '未定'

# 会社略称列を追加
df['会社略称'] = df['所属会社(内定先会社)'].apply(get_company_abbr)

# 社員/内定者の区分を統一
df['区分'] = df['内定者/社員'].str.strip()

# 参加者のみフィルタリング
participants = df[df['Bridge2026の参加可否'] == '参加'].copy()

# 会社略称と区分でグループ化してカウンター用辞書を作成
counters = {}

# Bridge IDを割り当てる関数
def assign_bridge_id(row):
    company = row['会社略称']
    category = row['区分']
    
    # キーを作成
    key = f"{company}_{category}"
    
    # カウンターを初期化または取得
    if key not in counters:
        counters[key] = 1
    else:
        counters[key] += 1
    
    # IDを生成（2桁ゼロパディング）
    bridge_id = f"{company}_{category}_{counters[key]:02d}"
    
    return bridge_id

# 参加者にBridge IDを割り当て
participants['新Bridge ID'] = participants.apply(assign_bridge_id, axis=1)

# 元のデータフレームに新しいBridge IDを反映
df['新Bridge ID'] = ''
df.loc[participants.index, '新Bridge ID'] = participants['新Bridge ID']


# 統計情報を表示
print("=" * 60)
print("Bridge ID 割り当て完了")
print("=" * 60)
print(f"\n総レコード数: {len(df)}")
print(f"参加者数: {len(participants)}")
print(f"不参加者数: {len(df[df['Bridge2026の参加可否'] == '不参加'])}")

print("\n【会社別・区分別の参加者数】")
print("-" * 60)

# 会社と区分でグループ化して集計
summary = participants.groupby(['会社略称', '区分']).size().reset_index(name='人数')
for _, row in summary.iterrows():
    print(f"{row['会社略称']:<10} {row['区分']:<6} : {row['人数']:3d}人")

Bridge ID 割り当て完了

総レコード数: 569
参加者数: 192
不参加者数: 377

【会社別・区分別の参加者数】
------------------------------------------------------------
SEC        内定者    :  55人
SEC        社員     :   8人
SGC        内定者    :  15人
SGC        社員     :   5人
SIE        内定者    :  16人
SIE        社員     :   7人
SSS        内定者    :  69人
SSS        社員     :   5人
未定         内定者    :  11人
未定         社員     :   1人


In [18]:
df.drop(columns=['タイムスタンプ', 'メールアドレス','開催にあたり、ご意見、ご希望などございましたらご自由にお書きください。(今後の運営の参考にさせていただきます)\r\n\r\nまた、疑問点やご不明な点がございましたら、お気軽にこちらのメールアドレスまでお問い合わせください。折り返しご連絡いたします。\r\nsony.bridge2026@gmail.com'], inplace=True)

In [19]:
# dfを「参加」の行のみ残す（上書き）
df = df[df['Bridge2026の参加可否'] == '参加'].copy()
df.head()
print(len(df))

192


In [20]:
df["Bridge ID"] = df["Bridge ID"].astype(int).astype(str).str.zfill(4)
df.head()

,お名前(漢字),お名前(フリガナ),Bridge2026の参加可否,内定者/社員,所属会社(内定先会社),Bridge ID,会社略称,区分,新Bridge ID
7,森大騎,モリダイキ,参加,社員,ソニー株式会社(SEC),0001,SEC,社員,SEC_社員_01
13,GU XIANGYU,コショウウ,参加,内定者,ソニーセミコンダクタソリューションズ株式会社(SSS),0002,SSS,内定者,SSS_内定者_01
14,木内絢美,キウチアヤミ,参加,内定者,ソニー株式会社(SEC),0003,SEC,内定者,SEC_内定者_01
17,榎本 優喜,エノモト ユウキ,参加,社員,ソニーセミコンダクタソリューションズ株式会社(SSS),0004,SSS,社員,SSS_社員_01
18,石井大哉,イシイダイヤ,参加,社員,ソニーグループ株式会社(SGC),0005,SGC,社員,SGC_社員_01


In [21]:
# 結果を新しいCSVファイルに保存
output_file = '../data/名簿一覧_Bridge_ID割当済_最終版.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')